In [39]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math
import urllib
import requests
from requests import ConnectionError, ReadTimeout

In [40]:
class Request(object):
    def __init__(self, request_session):
        self.request_session = request_session
    
    def get(self, url, headers={'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }, params={}, timeout=10, allow_redirects=False):
        try:
            print(url)
            headers['Host'] = urllib.request.splithost(urllib.request.splittype(url)[1])[0]
            res = self.request_session.get(url, headers=headers, params=params, timeout=timeout, allow_redirects=allow_redirects)
        except ConnectionError as ce:
            print('ConnectionError: ' + str(ce))
            return get(self, url=url, headers=headers, params=params, timeout=timeout, allow_redirects=allow_redirects)
        except ReadTimeout as rte:
            print('ReadTimeout: ' + str(rte))
            return get(self, url=url, headers=headers, params=params, timeout=timeout, allow_redirects=allow_redirects)

        return res
    
    def post(self, url, data, headers={'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }, timeout=10, allow_redirects=False):
        try:
            headers['host'] = urllib.request.splithost(urllib.request.splittype(url)[1])[0]
            res = self.request_session.post(url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ConnectionError as ce:
            print('ConnectionError: ' + str(ce))
            return post(self, url=url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ReadTimeout as rte:
            print('ReadTimeout: ' + str(rte))
            return post(self, url=url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)

        return res

    def get_cookie_dict(self):
        return requests.utils.dict_from_cookiejar(self.request_session.cookies)

## Initial Cookies

In [54]:
url = "https://moodle.kent.ac.uk/2017/course/view.php"
course_id = '2105'
querystring = {"id":course_id}

headers = {
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    'accept-language': "zh-CN,zh;q=0.9",
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'cache-control': "max-age=0,no-cache",
    'cookie': "MoodleSessionkm2017=kqlic5gpke61ne7qubehp7q1rh; optimizelyEndUserId=oeu1576805091684r0.5280257672419764; optimizelySegments=%7B%22700332969%22%3A%22direct%22%2C%22703200875%22%3A%22false%22%2C%22704334772%22%3A%22gc%22%7D; optimizelyBuckets=%7B%7D; _gcl_au=1.1.91719409.1576805122; _hjid=ad8026ec-1d07-4d45-bc5d-b0c8aa8841c1; _ga=GA1.3.1786459308.1576832247; _gid=GA1.3.1328829002.1577758071; SimpleSAMLSessionID=6eecfc0413e5d07070e7466581f16621; SimpleSAMLAuthToken=_b02b7446e53ee65aed98ea8c86a6b0163a2651196f",
    'referer': "https://moodle.kent.ac.uk/2017/my/index.php",
    'sec-fetch-mode': "navigate",
    'sec-fetch-site': "same-site",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.70 Safari/537.36",
    'Host': "moodle.kent.ac.uk",
    'Connection': "keep-alive"
    }

req = Request(requests.Session())
response = req.get(url, headers, querystring, 40, False)

print(response.headers)

while ('Location' in response.headers):
    headers['Referer'] = url
    url = response.headers.get('Location')

    response = req.get(url, headers=headers, params=querystring, timeout=40, allow_redirects=False)
    print(response.headers)

https://moodle.kent.ac.uk/2017/course/view.php
{'Server': 'nginx/1.17.7', 'Date': 'Tue, 31 Dec 2019 05:49:10 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Powered-By': 'PHP/7.2.26', 'Content-Language': 'en', 'Content-Script-Type': 'text/javascript', 'Content-Style-Type': 'text/css', 'X-UA-Compatible': 'IE=edge', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0, no-transform', 'Pragma': 'no-cache', 'Expires': 'Mon, 20 Aug 1969 09:23:00 GMT', 'Last-Modified': 'Tue, 31 Dec 2019 05:49:10 GMT', 'X-Cache-Status': 'MISS', 'Strict-Transport-Security': 'max-age=31536000', 'Content-Encoding': 'gzip'}


In [55]:
anchor = '"sesskey":"'
start = response.text.find(anchor)
end = response.text.find('"', start + len(anchor))
sesskey = response.text[start+ len(anchor):end]

In [56]:
url = 'https://moodle.kent.ac.uk/2017/blocks/panopto/panopto_content.php'
headers['Referer'] = url
headers['cookie'] = 'MoodleSessionkm2017=kqlic5gpke61ne7qubehp7q1rh; optimizelyEndUserId=oeu1576805091684r0.5280257672419764; optimizelySegments=%7B%22700332969%22%3A%22direct%22%2C%22703200875%22%3A%22false%22%2C%22704334772%22%3A%22gc%22%7D; optimizelyBuckets=%7B%7D; _gcl_au=1.1.91719409.1576805122; _hjid=ad8026ec-1d07-4d45-bc5d-b0c8aa8841c1; _ga=GA1.3.1786459308.1576832247; _gid=GA1.3.1328829002.1577758071; SimpleSAMLSessionID=6eecfc0413e5d07070e7466581f16621; SimpleSAMLAuthToken=_b02b7446e53ee65aed98ea8c86a6b0163a2651196f; _gat_kbtracker=1; _gat_kentmoodle=1'
headers['origin'] = 'https://moodle.kent.ac.uk'
headers['accept'] = '*/*'
post_form = {'sesskey':sesskey, 'courseid':course_id}
response = req.post(url, data=post_form, headers=headers)

In [57]:
response

<Response [200]>

In [59]:
anchor = 'href=\'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx'

start = 0

video_hrefs = []
while start >= 0:
    start = response.text.find(anchor, start)
    if start == -1:
        break
    end = response.text.find('\'', start + len(anchor))
    video_hrefs.append(response.text[start+6:end])
    start = end


In [60]:
video_hrefs

['https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=6f065f8f-d9e8-4182-a23e-bce07a430c4b',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=c01dd0ed-e15b-433d-8790-e29b33c4cc01',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=39e46273-af7f-4911-9614-c27c7b030557',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=b52916e0-bdc2-4b0d-8a01-f6c1368a609b',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=0f646b59-3eea-4798-9974-1763ea5ff3b9',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=378dec0a-60ec-43b3-a189-7f87ec8db401',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=27ce9c91-b12f-4672-831a-4ccaeedb408a',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=1af32c9e-e0a1-4539-b0d5-11bb2ecb194f',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=18af3863-652b-480f-a5f6-dd99bc579ccd',
 'https://kent.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=96ec8713-21b2-40a8-9d13-e9f9b48e2e5b',


In [133]:
headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-encoding': 'gzip, deflate, br',
  'accept-language': 'zh-CN,zh;q=0.9',
  'cache-control': 'max-age=0',
  'content-length': '15',
  'content-type': 'application/x-www-form-urlencoded',
  'cookie': '_ga=GA1.2.1491957841.1577065851; UserSettings=AnonymousUserID=7664f113-c2fb-471e-be42-ab2c001e726b&LastLoginMembershipProvider=Moodle; .ASPXAUTH=BB39652F8F3ED610AAF87FCD05D4868EF8DC8E2B6D93AC429CC8CB20387F6B85A888D5C12EE00BA22AAA530B8B36BFAB799B1D0AC7A8855AF1E6892CD41B0E9C2495342451B3F7A9A0ED698DDBFBFA393A135F0FA3ECC68F89437BFFC1A5D2E0112925E2BA60A1C4478B54B1FE202013984566E7322CDFC5160B2E08A3194F8FC2C20CD42D3DADDC8F9369D585B8DFE508781296F5F1A28F5E5FB91C7C1ED4A041CBB36A7B745CDB84DD8CA3A3DA1EF8C78B171A2FB086A6956A8E4576858DABD18B9676C334FD88BE013347D3044929C3DF043A5045570D2DC277D01BC31FA7CB8291A32EFEF0488935A9C2A9034F1F; _gid=GA1.2.1328811804.1577758279; unified\wbl={"defaultVolume":100}; sandboxCookie=13222246741.4087; csrfToken=GEkz8%2fUTJ1M1VdCV5nPqcKN0DEOtZrqCGei6y8%2fKuZG3qClhzN1iPZ6psgKkKL3VZclX8ckP9gEwocPBEWYFi3FgX3I%2flfRLKiZhh8Bm9Vy%2fiT%2f5vcebT1DxKDg9cm0hfieQgoNbXjo3ssoyFLDe955okLqErKs0RHvjHyktTzwabqkswFweFU%2bcaCKLtdB1Dius%2b3QxIUKB7nO8ZqDxysFUIZQuAyOnWwGAyqnSBtsDIKUDRPBXkxG2q0h1MvZE',
  'origin': 'https://moodle.kent.ac.uk',
  'referer': 'https://moodle.kent.ac.uk/2017/course/view.php?id=2105',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'cross-site',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.70 Safari/537.36'
}

In [125]:
response = req.post(video_hrefs[10], data={'instance':'Moodle'}, headers=headers)
response

<Response [200]>

In [134]:
for href in video_hrefs:
    start = href.find('id=')
    index_str = href[start+3:]

    url = f'https://kent.cloud.panopto.eu/Panopto/Podcast/Download/{index_str}.mp4?mediaTargetType=videoPodcast'

    response = req.get(url, headers=headers, timeout=400, allow_redirects=False)
    while ('Location' in response.headers):
        headers['Referer'] = url
        url = response.headers.get('Location')

        response = req.get(url, timeout=400, allow_redirects=False)
        print(response.headers)

    start = response.headers['Content-Disposition'].find('filename=')
    file_name = response.headers['Content-Disposition'][start+len('filename='):]
    with open('Kent_/'+file_name, 'wb') as mp4:
        mp4.write(response.content)

https://kent.cloud.panopto.eu/Panopto/Podcast/Download/6f065f8f-d9e8-4182-a23e-bce07a430c4b.mp4?mediaTargetType=videoPodcast


KeyError: 'content-disposition'

In [136]:
response

<Response [502]>